# Objective
The purpose is to create a demonstable prototype that mines purchase data and predicts categories similar to the input.
For elucidations sake we will divide the summary problem into sub problems.
*  Problem 1 -Given item A predict item B which is most associated through purchase patterns.


# Packages required
* Pandas for data frame
* Numpy for arrays

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

# Problem 1

# Import Data

In [2]:
data = pd.read_csv('../data/1LakhFMCGSalesWithCategory.csv', names = ['BillId','ItemId','ItemName','Level1','Level2','Level3','Level4','Level5','Level6'] )


In [3]:
#Dummy for pivot table
data['dummy'] = 1

In [25]:
data.loc[data['ItemId'] == 30279]

,BillId,ItemId,ItemName,Level1,Level2,Level3,Level4,Level5,Level6,dummy
10468,1273363,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
10494,1273980,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
10641,1274914,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
12312,1282382,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
12703,1289995,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
12853,141157,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
12881,1284316,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
13271,1283755,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
13286,142348,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1
13293,142723,30279,COLGATE PASTE STRONG TEETH 30GM,FMCG,FMCG NON FOOD,PERSONAL CARE,ORAL CARE,TOOTHPASTE,FRESHNESS,1


# Items Index

In [31]:
names = data[['ItemName', 'ItemId', 'Level5','Level6']]
names = names.drop_duplicates(subset = 'ItemName')
names.head()

,ItemName,ItemId,Level5,Level6
0,GULABARI ROSE GLOW CLEANSER 100ML,52344,FACE WASH,CLEANSING
1,J&J BABY WIPES 80PCS,59909,HYGIENE,WIPES
2,SAT ISABGOL 100GM,103829,CHURAN,NaN
3,COLGATE GEL MAXFRESH RED 150GM,30225,TOOTHPASTE,GEL
4,PATANJALI DANT KANTI MEDI ORAL GEL 100GM,91629,TOOTHPASTE,GEL


In [28]:
names.shape


(3106, 2)

In [38]:
names.to_csv('../data/namesWithLevels.csv')

# Pivot Table

In [5]:
matrix = data.pivot_table(values='dummy',index ='BillId', columns ='ItemId')


In [6]:
matrix_dummy = matrix.copy().fillna(0)

# Skewed Jaccardian
We define a jaccardian as intersection over union. A skewed Jaccardian is intersection over set A

### method from http://na-o-ys.github.io/others/2015-11-07-sparse-vector-similarities.html

In [7]:
sparse_matrix = sparse.csc_matrix(matrix_dummy)

In [8]:
type(sparse_matrix)

scipy.sparse.csc.csc_matrix

In [9]:
def jaccard_similarities(mat):
    cols_sum = mat.getnnz(axis=0)
    ab = mat.T * mat

    # for rows
    aa = np.repeat(cols_sum, ab.getnnz(axis=0))
    # for columns
    bb = cols_sum[ab.indices]

    similarities = ab.copy()
    similarities.data /= (aa + bb - ab.data)

    return similarities

In [10]:
jaccard_similarities =  jaccard_similarities(sparse_matrix)

In [11]:
jaccardian = pd.DataFrame(jaccard_similarities.toarray(), index = matrix.columns,columns = matrix.columns)

In [12]:
jaccardian.head()

ItemId,9254,9277,10377,10380,10381,10387,10390,10810,10813,10815,...,125123,125126,125129,125136,127187,127214,127244,127332,127444,127464
ItemId,,,,,,,,,,,,,,,,,,,,,
9254,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9277,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10377,0.0,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10380,0.0,0.0,0.0,1.000000,0.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10381,0.0,0.0,0.0,0.333333,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
df = pd.DataFrame(np.triu(jaccardian, 1), columns=jaccardian.columns, index=jaccardian.index)


In [67]:
filtered = df[df[df < 1] > 0.3].stack()

In [69]:
filtered


ItemId  ItemId
10380   10381     0.333333
11364   11369     0.500000
17205   110442    0.333333
17496   21060     0.500000
17524   17535     0.333333
17537   94894     0.333333
18774   21042     0.333333
19510   54444     0.500000
20285   20287     0.500000
20957   45998     0.333333
        68922     0.500000
21004   48131     0.500000
21035   21046     0.500000
21037   54788     0.500000
28899   28920     0.333333
29526   30344     0.333333
29828   37694     0.500000
31675   31676     0.400000
33545   100171    0.333333
        115870    0.500000
        117977    0.500000
34916   63179     0.333333
37586   40814     0.333333
        40815     0.500000
        69372     0.333333
40316   48390     0.333333
40342   54842     0.500000
41052   123365    0.500000
45998   68922     0.500000
48365   68939     0.333333
                    ...   
54725   98703     0.500000
        98705     0.666667
54727   98705     0.333333
54787   54813     0.333333
57143   83062     0.333333
61652   85760

# End Experiment

# Final Function

In [13]:
# Final Function
# Takes title, userId and returns to 25 sorted movie names on est
def reco(idx):

    similar_itemids = jaccardian.loc[idx].to_frame(name=None)
    topItemIds = similar_itemids.sort_values(by =[idx],ascending = False).head(26)
    topItemNames = topItemIds.merge(names, how ='left', on = 'ItemId' )

    
    return topItemNames.head(10)



In [66]:
reco(115445)

,ItemId,115445,ItemName,Level5,Level6
0,115445,1.000000,TOPAZ BLADE 1,SHAVING,BLADE AND CRT
1,98931,0.849315,RAZOR,SHAVING,RAZOR
2,123223,0.009009,WHISPER ULTRA 8 PADS,SANITARY NAPKINS,NaN
3,65300,0.002857,LACTOGEN 1 REFILL 400GM,MILK POWDER,STAGE 1
4,35436,0.002193,DETTOL SOAP ORIGINAL 50GM,SOAP,BAR ANTI BACTERIAL
5,9254,0.000000,555 DETERGENT POWDER 500GM,DETERGENT POWDER,CLEANING & FRAGRANCE
6,82792,0.000000,NIVEA CREAM ALOE 200ML,CREAM,ALOEVERA
7,82793,0.000000,NIVEA CREAM BLUE 100ML,CREAM,MOISTURISING
8,82795,0.000000,NIVEA CREAM BLUE 200ML,CREAM,MOISTURISING
9,82797,0.000000,NIVEA CREAM BLUE 20ML,CREAM,MOISTURISING


In [65]:
reco(54725)

,ItemId,54725,ItemName,Level5,Level6
0,54725,1.000000,HIMALAYA TOOTHPASTE SPARKLING WHITE 100GM,TOOTHPASTE,WHITENING
1,98705,0.666667,RASNA GUAVA 500GM,CONCENTRATE POWDER,GUAVA
2,63734,0.500000,KISSAN SQUASH MANGO 700ML,CONCENTRATE LIQUID,MANGO
3,54727,0.500000,HIMALAYA TOOTHPASTE SPARKLING WHITE 350GM,TOOTHPASTE,WHITENING
4,98702,0.500000,RASNA APPLE 500GM,CONCENTRATE POWDER,APPLE
5,98703,0.500000,RASNA COMBO PACK 2+1 300GM,CONCENTRATE POWDER,COMBO
6,63735,0.500000,KISSAN SQUASH MIX FRUIT 750ML,CONCENTRATE LIQUID,MIX FRUIT
7,54721,0.200000,HIMALAYA TOOTHPASTE COMPLETE CARE 175GMS,TOOTHPASTE,HERBAL
8,24854,0.142857,CANDY COP TOOTHPASTE 70GM,TOOTHPASTE,KID'S
9,54726,0.111111,HIMALAYA TOOTHPASTE SPARKLING WHITE 175GM,TOOTHPASTE,WHITENING


In [34]:
reco(35436)

,ItemId,35436,ItemName,Level5,Level6
0,35436,1.000000,DETTOL SOAP ORIGINAL 50GM,SOAP,BAR ANTI BACTERIAL
1,30279,0.030521,COLGATE PASTE STRONG TEETH 30GM,TOOTHPASTE,FRESHNESS
2,87473,0.015652,ORAL B TB SHINE CLEAN SOFT,TOOTH BRUSH,GENERAL
3,90759,0.012177,PARACHUTE FT 30ML,OIL,COCONUT
4,100796,0.011521,RIN ADVANCE BAR 86GM,BAR,CLEANING & FRAGRANCE
5,92863,0.010025,PEPSODENT BRUSH,TOOTH BRUSH,GENERAL
6,33609,0.009950,DABUR SARSON AMLA HAIR OIL 50ML,OIL,AMLA
7,30212,0.009328,COLGATE GEL MAX RED 28GM,TOOTHPASTE,GEL
8,87387,0.007905,ORAL B TB FRESH CLEAN,TOOTH BRUSH,GENERAL
9,90987,0.007576,PARLE AMERICANA BUTTER COOKIES,COOKIES,BUTTER BAKED


In [35]:
reco(30279)

,ItemId,30279,ItemName,Level5,Level6
0,30279,1.000000,COLGATE PASTE STRONG TEETH 30GM,TOOTHPASTE,FRESHNESS
1,87473,0.058306,ORAL B TB SHINE CLEAN SOFT,TOOTH BRUSH,GENERAL
2,87387,0.042254,ORAL B TB FRESH CLEAN,TOOTH BRUSH,GENERAL
3,35436,0.030521,DETTOL SOAP ORIGINAL 50GM,SOAP,BAR ANTI BACTERIAL
4,28670,0.030496,CLASSIC TB SUPREME SUPER SOFT,TOOTH BRUSH,GENERAL
5,30423,0.020408,COLGATE TB CIBACA 123 SOFT,TOOTH BRUSH,GENERAL
6,87420,0.015815,ORAL B TB 123 CAVITY DEFENSE SOFT,TOOTH BRUSH,GENERAL
7,30425,0.015019,COLGATE TB EXTRA CLEAN MEDIUM,TOOTH BRUSH,GENERAL
8,105689,0.014760,SENSODYNE TOOTH BRUSH SENSITIVE,TOOTH BRUSH,SENSTIVE
9,87432,0.014320,ORAL B TB CAVITY DEFENSE 123 BLACK MEDI,TOOTH BRUSH,GENERAL


In [36]:
reco(87432)

,ItemId,87432,ItemName,Level5,Level6
0,87432,1.000000,ORAL B TB CAVITY DEFENSE 123 BLACK MEDI,TOOTH BRUSH,GENERAL
1,30206,0.021898,COLGATE GEL MAX BLUE 80GM,TOOTHPASTE,GEL
2,22705,0.019608,BRITANNIA BISC NUTRI CHOICE OATS 150GM,COOKIES,OATS
3,87387,0.019231,ORAL B TB FRESH CLEAN,TOOTH BRUSH,GENERAL
4,30324,0.018868,COLGATE PASTE VISIBLE WHITE MINT 100GM,TOOTHPASTE,WHITENING
5,30279,0.014320,COLGATE PASTE STRONG TEETH 30GM,TOOTHPASTE,FRESHNESS
6,105680,0.013423,SENSODYNE RAPID RELIEF 40GM,TOOTHPASTE,SENSTIVE
7,30212,0.012500,COLGATE GEL MAX RED 28GM,TOOTHPASTE,GEL
8,17009,0.011494,AVENTS TEATS 0M+ NIPPLE,ACCESSORIES,NIPPLE
9,94798,0.011364,PONDS FACE WASH MENZ POLLUTION OUT 100GM,FACE WASH,CLEANSING


In [22]:
reco(22705)

,ItemId,22705,ItemName
0,22705,1.000000,BRITANNIA BISC NUTRI CHOICE OATS 150GM
1,94798,0.055556,PONDS FACE WASH MENZ POLLUTION OUT 100GM
2,79143,0.047619,MYSTIQUE NAPKIN TABLE GRACE 90 SHEETS
3,22707,0.045455,BRITANNIA BISC NUTRI CHOICE RAGI 150GM
4,91129,0.031250,PARLE BISC PINEAPPLE CREAM 50GM
5,81067,0.029412,NESTLE MILKY BAR CHOO CLASSIC 11GM
6,30309,0.028571,COLGATE PASTE TOTAL CHARCOAL D 140GM
7,30324,0.027027,COLGATE PASTE VISIBLE WHITE MINT 100GM
8,79146,0.024390,MYSTIQUE WET TISSUE WIPES DRY 20SHEETS
9,87432,0.019608,ORAL B TB CAVITY DEFENSE 123 BLACK MEDI
